In [1]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = './credentials/google_application_credentials.json'

In [2]:
from credentials import api_keys

In [1]:
import io
import requests
import pandas as pd
from sklearn import metrics
from google.cloud import vision

In [2]:
df = pd.read_json('./data/dev_seen.jsonl', lines=True)
df = df.drop(columns=['text'])
df['img'] = df['img'].apply(lambda x: './data/' + x)
print(df)

        id                   img  label
0     8291  ./data/img/08291.png      1
1    46971  ./data/img/46971.png      1
2     3745  ./data/img/03745.png      1
3    83745  ./data/img/83745.png      1
4    80243  ./data/img/80243.png      1
..     ...                   ...    ...
495  83675  ./data/img/83675.png      0
496  37198  ./data/img/37198.png      0
497  48670  ./data/img/48670.png      0
498   9863  ./data/img/09863.png      0
499  97320  ./data/img/97320.png      0

[500 rows x 3 columns]


## Google Cloud Vision API

In [5]:
def google_vision_api(path):
    """Google Cloud Vision API"""
    client = vision.ImageAnnotatorClient()

    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    response = client.safe_search_detection(image=image)
    # print(response)
    safe = response.safe_search_annotation

    if response.error.message:
        raise Exception(response.error.message)

    if safe.adult > 3 or safe.spoof > 3 or safe.medical > 3 or safe.violence > 3 or safe.racy > 3:
        print('Image: {}, UNSAFE'.format(path))
        return 1
    else:
        print('Image: {}, SAFE'.format(path))
        return 0

In [6]:
df_google = df.copy()
df_google['pred_gv'] = df_google['img'].apply(google_vision_api)
df_google.to_csv('./results/df_google.csv')
print(df_google)

Image: ./data/img/08291.png, UNSAFE
Image: ./data/img/46971.png, UNSAFE
Image: ./data/img/03745.png, UNSAFE
Image: ./data/img/83745.png, UNSAFE
Image: ./data/img/80243.png, UNSAFE
Image: ./data/img/05279.png, UNSAFE
Image: ./data/img/01796.png, UNSAFE
Image: ./data/img/53046.png, SAFE
Image: ./data/img/82301.png, UNSAFE
Image: ./data/img/31752.png, UNSAFE
Image: ./data/img/27635.png, UNSAFE
Image: ./data/img/80597.png, UNSAFE
Image: ./data/img/45368.png, UNSAFE
Image: ./data/img/17963.png, SAFE
Image: ./data/img/53968.png, UNSAFE
Image: ./data/img/10749.png, UNSAFE
Image: ./data/img/25149.png, UNSAFE
Image: ./data/img/87520.png, UNSAFE
Image: ./data/img/89071.png, UNSAFE
Image: ./data/img/09563.png, UNSAFE
Image: ./data/img/72048.png, UNSAFE
Image: ./data/img/49826.png, UNSAFE
Image: ./data/img/26453.png, UNSAFE
Image: ./data/img/12650.png, UNSAFE
Image: ./data/img/02568.png, UNSAFE
Image: ./data/img/83954.png, SAFE
Image: ./data/img/24316.png, UNSAFE
Image: ./data/img/62035.png, UNSAF

In [3]:
print('Classification Report for Google Cloud Vision API:')
df_google = pd.read_csv('./results/df_google.csv')
target_names = ['safe', 'not-safe']
print(metrics.classification_report(df_google['label'], df_google['pred_gv'], target_names=target_names, digits=4))

Classification Report for Google Cloud Vision API:
              precision    recall  f1-score   support

        safe     0.3934    0.0949    0.1529       253
    not-safe     0.4784    0.8502    0.6122       247

    accuracy                         0.4680       500
   macro avg     0.4359    0.4725    0.3826       500
weighted avg     0.4354    0.4680    0.3798       500



In [ ]:
def google_vision_api_without_spoof(path):
    """Google Cloud Vision API"""
    client = vision.ImageAnnotatorClient()

    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    response = client.safe_search_detection(image=image)
    # print(response)
    safe = response.safe_search_annotation

    if response.error.message:
        raise Exception(response.error.message)

    if safe.adult > 3 or safe.medical > 3 or safe.violence > 3 or safe.racy > 3:
        print('Image: {}, UNSAFE'.format(path))
        return 1
    else:
        print('Image: {}, SAFE'.format(path))
        return 0

In [7]:
df_google_2 = df.copy()
df_google_2['pred_gv'] = df_google_2['img'].apply(google_vision_api_without_spoof)
df_google_2.to_csv('./results/df_google_2.csv')
print(df_google_2)

Image: ./data/img/08291.png, SAFE
Image: ./data/img/46971.png, SAFE
Image: ./data/img/03745.png, SAFE
Image: ./data/img/83745.png, UNSAFE
Image: ./data/img/80243.png, SAFE
Image: ./data/img/05279.png, SAFE
Image: ./data/img/01796.png, SAFE
Image: ./data/img/53046.png, SAFE
Image: ./data/img/82301.png, SAFE
Image: ./data/img/31752.png, SAFE
Image: ./data/img/27635.png, SAFE
Image: ./data/img/80597.png, SAFE
Image: ./data/img/45368.png, UNSAFE
Image: ./data/img/17963.png, SAFE
Image: ./data/img/53968.png, SAFE
Image: ./data/img/10749.png, SAFE
Image: ./data/img/25149.png, UNSAFE
Image: ./data/img/87520.png, SAFE
Image: ./data/img/89071.png, SAFE
Image: ./data/img/09563.png, SAFE
Image: ./data/img/72048.png, SAFE
Image: ./data/img/49826.png, SAFE
Image: ./data/img/26453.png, SAFE
Image: ./data/img/12650.png, UNSAFE
Image: ./data/img/02568.png, SAFE
Image: ./data/img/83954.png, SAFE
Image: ./data/img/24316.png, SAFE
Image: ./data/img/62035.png, SAFE
Image: ./data/img/65342.png, SAFE
Image:

In [4]:
print('Classification Report for Google Cloud Vision API without Spoof:')
df_google_2 = pd.read_csv('./results/df_google_2.csv')
target_names = ['safe', 'not-safe']
print(metrics.classification_report(df_google_2['label'], df_google_2['pred_gv'], target_names=target_names, digits=4))

Classification Report for Google Cloud Vision API without Spoof:
              precision    recall  f1-score   support

        safe     0.5143    0.9249    0.6610       253
    not-safe     0.5778    0.1053    0.1781       247

    accuracy                         0.5200       500
   macro avg     0.5460    0.5151    0.4195       500
weighted avg     0.5457    0.5200    0.4224       500



## Yahoo Open NSFW

In [5]:
print('Classification Report for Yahoo Open NSFW:')
df_yahoo = pd.read_csv('./results/df_yahoo.csv')
df_yahoo['pred_yh'] = df_yahoo['pred_yh_value'].apply(lambda x: 1 if x >= 0.5 else 0 )
target_names = ['safe', 'not-safe']
print(metrics.classification_report(df_yahoo['label'], df_yahoo['pred_yh'], target_names=target_names, digits=4))

Classification Report for Yahoo Open NSFW:
              precision    recall  f1-score   support

        safe     0.5051    0.9802    0.6667       253
    not-safe     0.4444    0.0162    0.0312       247

    accuracy                         0.5040       500
   macro avg     0.4748    0.4982    0.3490       500
weighted avg     0.4751    0.5040    0.3528       500



## Clarifai NSFW API

In [8]:
from clarifai_grpc.channel.clarifai_channel import ClarifaiChannel
from clarifai_grpc.grpc.api import resources_pb2, service_pb2, service_pb2_grpc
from clarifai_grpc.grpc.api.status import status_pb2, status_code_pb2

stub = service_pb2_grpc.V2Stub(ClarifaiChannel.get_grpc_channel())
metadata = (('authorization', api_keys.CLARIFAI_API_KEY),)

In [9]:
def clarifai_api(path):
    with open(path, "rb") as f:
        file_bytes = f.read()

    request = service_pb2.PostModelOutputsRequest(
        # This is the model ID of a publicly available General model. 
        # You may use any other public or custom model ID.
        model_id='e9576d86d2004ed1a38ba0cf39ecb4b1',
        inputs=[
            resources_pb2.Input(
                data=resources_pb2.Data(
                    image=resources_pb2.Image(
                        base64=file_bytes
                    )
                )
            )
        ])
    response = stub.PostModelOutputs(request, metadata=metadata)

    if response.status.code != status_code_pb2.SUCCESS:
        raise Exception("Post model outputs failed, status: " + response.status.description)

    # for concept in response.outputs[0].data.concepts:
    #     print('%12s: %.2f' % (concept.name, concept.value))

    if response.outputs[0].data.concepts[1].value >= 0.4:
        print('Image: {}, Offensive: {:.2f}'.format(path, response.outputs[0].data.concepts[1].value))
    else:
        print('Image: {}, not-Offensive: {:.2f}'.format(path, response.outputs[0].data.concepts[1].value))

    return response.outputs[0].data.concepts[1].value

In [10]:
df_clarifai = df.copy()
df_clarifai['pred_cf_value'] = df_clarifai['img'].apply(clarifai_api)
df_clarifai.to_csv('./results/df_clarifai.csv')
print(df_clarifai)

Image: ./data/img/08291.png, not-Offensive: 0.01
Image: ./data/img/46971.png, not-Offensive: 0.00
Image: ./data/img/03745.png, not-Offensive: 0.03
Image: ./data/img/83745.png, not-Offensive: 0.01
Image: ./data/img/80243.png, not-Offensive: 0.06
Image: ./data/img/05279.png, not-Offensive: 0.04
Image: ./data/img/01796.png, not-Offensive: 0.00
Image: ./data/img/53046.png, not-Offensive: 0.00
Image: ./data/img/82301.png, not-Offensive: 0.00
Image: ./data/img/31752.png, not-Offensive: 0.00
Image: ./data/img/27635.png, not-Offensive: 0.00
Image: ./data/img/80597.png, not-Offensive: 0.01
Image: ./data/img/45368.png, not-Offensive: 0.05
Image: ./data/img/17963.png, not-Offensive: 0.01
Image: ./data/img/53968.png, not-Offensive: 0.00
Image: ./data/img/10749.png, not-Offensive: 0.01
Image: ./data/img/25149.png, not-Offensive: 0.00
Image: ./data/img/87520.png, not-Offensive: 0.11
Image: ./data/img/89071.png, not-Offensive: 0.00
Image: ./data/img/09563.png, not-Offensive: 0.00
Image: ./data/img/72

In [6]:
print('Classification Report for Clarifai NSFW API:')
df_clarifai = pd.read_csv('./results/df_clarifai.csv')
df_clarifai['pred_cf'] = df_clarifai['pred_cf_value'].apply(lambda x: 1 if x >= 0.5 else 0 )
target_names = ['safe', 'not-safe']
print(metrics.classification_report(df_clarifai['label'], df_clarifai['pred_cf'], target_names=target_names, digits=4))

Classification Report for Clarifai NSFW API:
              precision    recall  f1-score   support

        safe     0.5060    1.0000    0.6720       253
    not-safe     0.0000    0.0000    0.0000       247

    accuracy                         0.5060       500
   macro avg     0.2530    0.5000    0.3360       500
weighted avg     0.2560    0.5060    0.3400       500

/Users/andyma/anaconda3/envs/memes/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/andyma/anaconda3/envs/memes/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(a

## DeepAI API

In [6]:
def deep_ai_api(path):
    deep_ai_request = requests.post(
        'https://api.deepai.org/api/content-moderation',
        files={
            'image': open(path, 'rb'),
        },
        headers={'api-key': api_keys.DEEPAI_API_KEY}
    )
    nsfw_score = deep_ai_request.json()['output']['nsfw_score']
    print('Image: {}, nsfw_score: {:.2f}'.format(path, nsfw_score))
    return nsfw_score

In [7]:
df_deepai = df.copy()
df_deepai['pred_da_value'] = df_deepai['img'].apply(deep_ai_api)
df_deepai.to_csv('./results/df_deepai.csv')
print(df_deepai)

Image: ./data/img/08291.png, nsfw_score: 0.26
Image: ./data/img/46971.png, nsfw_score: 0.10
Image: ./data/img/03745.png, nsfw_score: 0.04
Image: ./data/img/83745.png, nsfw_score: 0.71
Image: ./data/img/80243.png, nsfw_score: 0.06
Image: ./data/img/05279.png, nsfw_score: 0.00
Image: ./data/img/01796.png, nsfw_score: 0.07
Image: ./data/img/53046.png, nsfw_score: 0.00
Image: ./data/img/82301.png, nsfw_score: 0.00
Image: ./data/img/31752.png, nsfw_score: 0.01
Image: ./data/img/27635.png, nsfw_score: 0.00
Image: ./data/img/80597.png, nsfw_score: 0.00
Image: ./data/img/45368.png, nsfw_score: 0.00
Image: ./data/img/17963.png, nsfw_score: 0.06
Image: ./data/img/53968.png, nsfw_score: 0.01
Image: ./data/img/10749.png, nsfw_score: 0.01
Image: ./data/img/25149.png, nsfw_score: 0.02
Image: ./data/img/87520.png, nsfw_score: 0.00
Image: ./data/img/89071.png, nsfw_score: 0.01
Image: ./data/img/09563.png, nsfw_score: 0.00
Image: ./data/img/72048.png, nsfw_score: 0.00
Image: ./data/img/49826.png, nsfw_

In [7]:
print('Classification Report for DeepAI API:')
df_deepai = pd.read_csv('./results/df_deepai.csv')
df_deepai['pred_da'] = df_deepai['pred_da_value'].apply(lambda x: 1 if x >= 0.5 else 0 )
target_names = ['safe', 'not-safe']
print(metrics.classification_report(df_deepai['label'], df_deepai['pred_da'], target_names=target_names, digits=4))

Classification Report for DeepAI API:
              precision    recall  f1-score   support

        safe     0.5040    0.9881    0.6676       253
    not-safe     0.2500    0.0040    0.0080       247

    accuracy                         0.5020       500
   macro avg     0.3770    0.4961    0.3378       500
weighted avg     0.3785    0.5020    0.3417       500



In [38]:
def deep_ai_api_object(path):
    deep_ai_request = requests.post(
        'https://api.deepai.org/api/content-moderation',
        files={
            'image': open(path, 'rb'),
        },
        headers={'api-key': api_keys.DEEPAI_API_KEY}
    )
    # print(deep_ai_request.json())
    # print(deep_ai_request.json()['output']['detections'])
    # print(len(deep_ai_request.json()['output']['detections']))
    objects = len(deep_ai_request.json()['output']['detections'])

    print('Image: {}, not-safe objects: {:>3}'.format(path, objects))
    return objects

In [39]:
df_deepai_objects = df.copy()
df_deepai_objects['pred_da_value'] = df_deepai_objects['img'].apply(deep_ai_api_object)
df_deepai_objects.to_csv('./results/df_deepai_objects.csv')
print(df_deepai_objects)

/img/92738.png, not-safe objects:   0
Image: ./data/img/53027.png, not-safe objects:   0
Image: ./data/img/27195.png, not-safe objects:   1
Image: ./data/img/43910.png, not-safe objects:   0
Image: ./data/img/05213.png, not-safe objects:   0
Image: ./data/img/73945.png, not-safe objects:   0
Image: ./data/img/56207.png, not-safe objects:   0
Image: ./data/img/68459.png, not-safe objects:   0
Image: ./data/img/63175.png, not-safe objects:   0
Image: ./data/img/82945.png, not-safe objects:   1
Image: ./data/img/34975.png, not-safe objects:   0
Image: ./data/img/61973.png, not-safe objects:   0
Image: ./data/img/16420.png, not-safe objects:   0
Image: ./data/img/73069.png, not-safe objects:   0
Image: ./data/img/47056.png, not-safe objects:   1
Image: ./data/img/56428.png, not-safe objects:   0
Image: ./data/img/54206.png, not-safe objects:   0
Image: ./data/img/93172.png, not-safe objects:   0
Image: ./data/img/56980.png, not-safe objects:   0
Image: ./data/img/79615.png, not-safe object

In [40]:
print('Classification Report for DeepAI API on Unsafe Content:')
df_deepai_objects = pd.read_csv('./results/df_deepai_objects.csv')
df_deepai_objects['pred_da'] = df_deepai_objects['pred_da_value'].apply(lambda x: 1 if x > 0 else 0 )
target_names = ['safe', 'not-safe']
print(metrics.classification_report(df_deepai_objects['label'], df_deepai_objects['pred_da'], target_names=target_names, digits=4))

Classification Report for DeepAI API on Unsafe Content:
              precision    recall  f1-score   support

        safe     0.5075    0.9368    0.6583       253
    not-safe     0.5152    0.0688    0.1214       247

    accuracy                         0.5080       500
   macro avg     0.5113    0.5028    0.3899       500
weighted avg     0.5113    0.5080    0.3931       500



## Amazon Rekognition

In [41]:
import boto3

In [28]:
def amz_reko_api(path):
    with open(path, "rb") as f:
        file_bytes = f.read()

    client=boto3.client('rekognition')

    response = client.detect_moderation_labels(Image={'Bytes': file_bytes}, MinConfidence=0)

    if len(response['ModerationLabels']) == 0:
        print('Image: {}, SAFE'.format(path))
        return 0

    highest_confidence = 0
    highest_name = ''
    for label in response['ModerationLabels']:
        if label['Confidence'] > highest_confidence:
            highest_name = label['Name']
            highest_confidence = label['Confidence']

    print('Image: {}, {}: {:.2f}'.format(path, highest_name, highest_confidence/100))
    return highest_confidence/100

    # print('Detected labels for ' + path)    
    # for label in response['ModerationLabels']:
    #     print (label['Name'] + ' : ' + str(label['Confidence']/100))
    #     print (label['ParentName'])
    # return len(response['ModerationLabels'])

In [29]:
df_amz = df.copy()
df_amz['pred_amz_value'] = df_amz['img'].apply(amz_reko_api)
df_amz.to_csv('./results/df_amz.csv')
print(df_amz)

Image: ./data/img/08291.png, SAFE
Image: ./data/img/46971.png, SAFE
Image: ./data/img/03745.png, SAFE
Image: ./data/img/83745.png, SAFE
Image: ./data/img/80243.png, SAFE
Image: ./data/img/05279.png, SAFE
Image: ./data/img/01796.png, SAFE
Image: ./data/img/53046.png, SAFE
Image: ./data/img/82301.png, SAFE
Image: ./data/img/31752.png, SAFE
Image: ./data/img/27635.png, SAFE
Image: ./data/img/80597.png, SAFE
Image: ./data/img/45368.png, SAFE
Image: ./data/img/17963.png, SAFE
Image: ./data/img/53968.png, SAFE
Image: ./data/img/10749.png, SAFE
Image: ./data/img/25149.png, SAFE
Image: ./data/img/87520.png, SAFE
Image: ./data/img/89071.png, SAFE
Image: ./data/img/09563.png, SAFE
Image: ./data/img/72048.png, SAFE
Image: ./data/img/49826.png, SAFE
Image: ./data/img/26453.png, SAFE
Image: ./data/img/12650.png, SAFE
Image: ./data/img/02568.png, Middle Finger: 0.85
Image: ./data/img/83954.png, SAFE
Image: ./data/img/24316.png, SAFE
Image: ./data/img/62035.png, SAFE
Image: ./data/img/65342.png, Weap

In [8]:
print('Classification Report for Amazon Rekognition:')
df_amz = pd.read_csv('./results/df_amz.csv')
df_amz['pred_amz'] = df_amz['pred_amz_value'].apply(lambda x: 1 if x >= 0.5 else 0 )
target_names = ['safe', 'not-safe']
print(metrics.classification_report(df_amz['label'], df_amz['pred_amz'], target_names=target_names, digits=4))

Classification Report for Amazon Rekognition:
              precision    recall  f1-score   support

        safe     0.5033    0.9170    0.6499       253
    not-safe     0.4615    0.0729    0.1259       247

    accuracy                         0.5000       500
   macro avg     0.4824    0.4949    0.3879       500
weighted avg     0.4826    0.5000    0.3910       500



In [60]:
def amz_reko_api_content(path):
    with open(path, "rb") as f:
        file_bytes = f.read()

    client=boto3.client('rekognition')

    response = client.detect_moderation_labels(Image={'Bytes': file_bytes})

    if len(response['ModerationLabels']) == 0:
        print('Image: {}, SAFE'.format(path))
        return 0

    for item in response['ModerationLabels']:
        if item['Name'] == 'Hate Symbols':
            print('Image: {}, Hateful'.format(path))
            return 1

    return 0

In [61]:
df_amz_hateful = df.copy()
df_amz_hateful['pred_amz_value'] = df_amz_hateful['img'].apply(amz_reko_api_content)
df_amz_hateful.to_csv('./results/df_amz_hateful.csv')
print(df_amz_hateful)

Image: ./data/img/08291.png, SAFE
Image: ./data/img/46971.png, SAFE
Image: ./data/img/03745.png, SAFE
Image: ./data/img/83745.png, SAFE
Image: ./data/img/80243.png, SAFE
Image: ./data/img/05279.png, SAFE
Image: ./data/img/01796.png, SAFE
Image: ./data/img/53046.png, SAFE
Image: ./data/img/82301.png, SAFE
Image: ./data/img/31752.png, SAFE
Image: ./data/img/27635.png, SAFE
Image: ./data/img/80597.png, SAFE
Image: ./data/img/45368.png, SAFE
Image: ./data/img/17963.png, SAFE
Image: ./data/img/53968.png, SAFE
Image: ./data/img/10749.png, SAFE
Image: ./data/img/25149.png, SAFE
Image: ./data/img/87520.png, SAFE
Image: ./data/img/89071.png, SAFE
Image: ./data/img/09563.png, SAFE
Image: ./data/img/72048.png, SAFE
Image: ./data/img/49826.png, SAFE
Image: ./data/img/26453.png, SAFE
Image: ./data/img/12650.png, SAFE
Image: ./data/img/83954.png, SAFE
Image: ./data/img/24316.png, SAFE
Image: ./data/img/62035.png, SAFE
Image: ./data/img/92058.png, SAFE
Image: ./data/img/58672.png, SAFE
Image: ./data/

In [62]:
print('Classification Report for Amazon Rekognition on Hate Content:')
df_amz_hateful = pd.read_csv('./results/df_amz_hateful.csv')
df_amz_hateful['pred_amz'] = df_amz_hateful['pred_amz_value'].apply(lambda x: 1 if x >= 0.5 else 0 )
target_names = ['safe', 'hateful']
print(metrics.classification_report(df_amz_hateful['label'], df_amz_hateful['pred_amz'], target_names=target_names, digits=4))

Classification Report for Amazon Rekognition on Hate Content:
              precision    recall  f1-score   support

        safe     0.5051    0.9802    0.6667       253
     hateful     0.4444    0.0162    0.0312       247

    accuracy                         0.5040       500
   macro avg     0.4748    0.4982    0.3490       500
weighted avg     0.4751    0.5040    0.3528       500



## Data Combination

In [9]:
df['pred_gv'] = df_google_2['pred_gv']
df['pred_yh'] = df_yahoo['pred_yh']
df['pred_cf'] = df_clarifai['pred_cf']
df['pred_da'] = df_deepai['pred_da']
df['pred_amz'] = df_amz['pred_amz']

In [10]:
df['pred_total'] = df['pred_gv'] + df['pred_yh'] + df_clarifai['pred_cf'] + df_deepai['pred_da'] + df_amz['pred_amz']
print(df)

        id                   img  label  pred_gv  pred_yh  pred_cf  pred_da  \
0     8291  ./data/img/08291.png      1        0        0        0        0   
1    46971  ./data/img/46971.png      1        0        0        0        0   
2     3745  ./data/img/03745.png      1        0        0        0        0   
3    83745  ./data/img/83745.png      1        1        1        0        1   
4    80243  ./data/img/80243.png      1        0        0        0        0   
..     ...                   ...    ...      ...      ...      ...      ...   
495  83675  ./data/img/83675.png      0        0        0        0        0   
496  37198  ./data/img/37198.png      0        0        0        0        0   
497  48670  ./data/img/48670.png      0        0        0        0        0   
498   9863  ./data/img/09863.png      0        0        0        0        0   
499  97320  ./data/img/97320.png      0        0        0        0        0   

     pred_amz  pred_total  
0           0          

In [12]:
df_pos = df[df.label.eq(1)]
df_pos_rows = df_pos.shape[0]
print(df_pos_rows)
df_false_neg = df_pos[df_pos.pred_total.eq(0)]
df_false_neg_rows = df_false_neg.shape[0]
print(df_false_neg_rows)
pass_rate = df_false_neg_rows / df_pos_rows
print(pass_rate)

247
207
0.8380566801619433


In [11]:
df_neg = df[df.label.eq(0)]
df_neg_rows = df_neg.shape[0]
print(df_neg_rows)
df_false_pos = df_neg[df_neg.pred_total.gt(0)]
df_false_pos_rows = df_false_pos.shape[0]
print(df_false_pos_rows)
false_rate = df_false_pos_rows / df_neg_rows
print(false_rate)

253
37
0.14624505928853754
